In [ ]:
## Clean Facebook Data

import pandas as pd
import os
import glob

path = r"C:\Users\where\my\data\store"
path_file = glob.glob(os.path.join(path, "*.xlsx"))

final_path = r"C:\Users\where\my\data\store\final"
os.makedirs(final_path, exist_ok=True)

agencies = {
    'agency01': {
        'business_id': '',
        'agency_id': '',
        'agency': '',
    },
    'agency02': {
        'business_id': '',
        'agency_id': '',
        'agency': '',
    },
}

## import advertiser from google sheet
SHEET_ID = 'SHEET_ID'
SHEET_NAME = 'SHEET_NAME'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
adver = pd.read_csv(url)

# blank dataframe for concat file
df_dpp = pd.DataFrame()
df_ag = pd.DataFrame()

for file in path_file:
    file_name = os.path.basename(file)
    for agency_name, agency_data in agencies.items():
        if agency_name in file_name:
            business_id = agency_data['business_id']
            agency_id = agency_data['agency_id']
            agency = agency_data['agency']

            df = pd.read_excel(file)

            if 'agegender' in file_name:
                col_check = ['Account ID', 'Account name', 'Campaign ID', 'Campaign name', 'Month', 'Gender', 'Age', 'Ad set ID', 'Ad set name', 'Ad ID', 'Ad name', 'Impressions', 'Currency', 'Amount spent', 'Link clicks', 'Page likes', '3-second video plays', 'Landing page views', 'Objective', 'App Installs', 'Meta leads', 'Mobile app adds to cart', 'Website adds to cart', 'Mobile app content views', 'Website content views', 'Mobile app purchases', 'Website purchases', 'Mobile app adds to cart conversion value', 'Website adds to cart conversion value', 'Mobile app purchases conversion value', 'Website purchases conversion value', 'Leads', 'Clicks (all)', 'Post engagement', 'Reporting starts', 'Reporting ends']
            elif 'performance' in file_name:
                col_check = ['Account ID', 'Account name', 'Campaign ID', 'Campaign name', 'Month', 'Platform', 'Device platform', 'Placement', 'Ad set ID', 'Ad set name', 'Ad ID', 'Ad name', 'Impressions', 'Currency', 'Amount spent', 'Link clicks', 'Page likes', '3-second video plays', 'Landing page views', 'Objective', 'App Installs', 'Meta leads', 'Mobile app adds to cart', 'Website adds to cart', 'Mobile app content views', 'Website content views', 'Mobile app purchases', 'Website purchases', 'Mobile app adds to cart conversion value', 'Website adds to cart conversion value', 'Mobile app purchases conversion value', 'Website purchases conversion value', 'Leads', 'Clicks (all)', 'Post engagement', 'Reporting starts', 'Reporting ends']

            # Rename columns if needed
            if 'Amount spent (THB)' in df.columns:
                df.rename(columns={'Amount spent (THB)': 'Amount spent'}, inplace=True)
            elif 'landing page view' in df.columns:
                df.rename(columns={'landing page view': 'Landing page views'}, inplace=True)

            # Check if file and variable have same columns
            if set(col_check).issubset(df.columns):
                extract_year = df['Month'].str.extract("([0-9]{4})")
                df = df[col_check]
                df.drop(df.columns[-2:], axis=1, inplace=True)
                df.insert(0, 'Business ID', business_id)
                df.insert(1, 'Media Platform', 'Facebook')
                df.insert(2, 'Agency ID', agency_id)
                df.insert(3, 'Agency', agency)
                df.insert(4, 'Year', extract_year)
                df['Month'] = df['Month'].str[5:7].astype(str).apply(lambda x: '0'+x if len(x) == 1 else x )

                ## Join data with google sheet
                df = pd.merge(df, adver, how='left', on='Account name')

                ## rename to lower and replace white spaces with underscore
                df.columns = map(str.lower, df.columns)
                df.columns = df.columns.str.replace(' ', '_')

            # 2 ifs are for changing data type
            if 'agegender' in file:
                col_str = ['business_id', 'media_platform', 'agency_id', 'agency', 'year', 'account_id', 'account_name', 'campaign_id', 'campaign_name', 'month', 'gender', 'age',
                            'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name', 'currency', 'objective']
                col_float = ['impressions', 'amount_spent', 'link_clicks', 'page_likes', '3-second_video_plays', 'landing_page_views', 'app_installs', 'meta_leads', 'mobile_app_adds_to_cart', 
                            'website_adds_to_cart', 'mobile_app_content_views', 'website_content_views', 'mobile_app_purchases', 'website_purchases', 'mobile_app_adds_to_cart_conversion_value', 
                            'website_adds_to_cart_conversion_value', 'mobile_app_purchases_conversion_value', 'website_purchases_conversion_value', 'leads', 'clicks_(all)', 'post_engagement']
                df[col_str] = df[col_str].astype(str).apply(lambda x: x.str.rstrip('.0'))
                df[col_float] = df[col_float].astype(float)

                df_ag = pd.concat([df_ag, df])
                df_ag.to_parquet(os.path.join(final_path, 'fb_age-gender_output.parquet'), index=False) 


            elif 'performance' in file:
                col_str = ['business_id', 'media_platform', 'agency_id', 'agency', 'year', 'account_id', 'account_name', 'campaign_id', 'campaign_name', 'month',
                            'platform', 'device_platform', 'placement', 'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name', 'currency', 'objective']
                col_float = ['impressions', 'amount_spent', 'link_clicks', 'page_likes', '3-second_video_plays', 'landing_page_views', 'app_installs', 'meta_leads', 'mobile_app_adds_to_cart', 
                            'website_adds_to_cart', 'mobile_app_content_views', 'website_content_views', 'mobile_app_purchases', 'website_purchases', 'mobile_app_adds_to_cart_conversion_value', 
                            'website_adds_to_cart_conversion_value', 'mobile_app_purchases_conversion_value', 'website_purchases_conversion_value', 'leads', 'clicks_(all)', 'post_engagement']
                df[col_str] = df[col_str].astype(str).apply(lambda x: x.str.rstrip('.0'))
                df[col_float] = df[col_float].astype(float)

                df_dpp = pd.concat([df_dpp, df])
                df_dpp.to_parquet(os.path.join(final_path, 'fb_dpp_output.parquet'), index=False) 

In [ ]:
## Convert FB add-on data to parquet file

path = r'C:\Users\my\facebook\add-on\data'
path_file = glob.glob(os.path.join(path, "*.csv"))

# Empty dataframe for concate files
dpp_df = pd.DataFrame()
agegender_df = pd.DataFrame()

for file in path_file:
    df = pd.read_csv(file)

    ## rename to lower and replace white spaces with underscore
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace(' ', '_')

    ## rename columns
    df.rename(columns={'campaign_objective': 'objective', 'account': 'account_name'}, inplace=True)

    ## convert data type
    if 'dpp' in file:
        col_str = ['account_id', 'account_name', 'campaign_id', 'campaign_name', 'objective', 'year', 'month', 'creative_object_type',
       'device_platform', 'placement', 'platform', 'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name']
        col_float = ['impressions', 'event_responses', 'outbound_clicks', 'leads', 'video_thruplay']

        df[col_str] = df[col_str].astype(str)
        df[col_float] = df[col_float].astype(float)
        
        dpp_df = pd.concat([dpp_df, df])

        ## Add 0 in the front of month number
        dpp_df['month'] = dpp_df['month'].str.zfill(2)

        ## because original 1 month file was splitted into 2 parts, So I decided use group by to remove duplicate values
        dpp_df = dpp_df.groupby(['account_id', 'account_name', 'campaign_id', 'campaign_name',
       'objective', 'year', 'month', 'creative_object_type',
       'device_platform', 'placement', 'platform', 'ad_set_id', 'ad_set_name',
       'ad_id', 'ad_name'], as_index=False)[col_float].sum()

        dpp_df.to_parquet(os.path.join(path_file, 'fb_add-on_dpp_output.parquet'), index=False)

    if 'agegender' in file:
        col_str = ['account_id', 'account_name', 'campaign_id', 'campaign_name', 'objective', 'year', 'month', 'creative_object_type',
       'age', 'gender', 'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name']
        col_float = ['impressions', 'event_responses', 'outbound_clicks', 'leads', 'video_thruplay']

        df[col_str] = df[col_str].astype(str)
        df[col_float] = df[col_float].astype(float)

        agegender_df = pd.concat([agegender_df, df])

        ## Add 0 in the front of month number
        agegender_df['month'] = agegender_df['month'].str.zfill(2)

        ## because original 1 month file was splitted into 2 parts, So I decided use group by to remove duplicate values
        agegender_df = agegender_df.groupby(['account_id', 'account_name', 'campaign_id', 'campaign_name',
       'objective', 'year', 'month', 'creative_object_type',
       'age', 'gender', 'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name'], as_index=False)[col_float].sum()

        agegender_df.to_parquet(os.path.join(path_file, 'fb_add-on_agegender_output.parquet'), index=False)

In [ ]:
par_path = r"C:\Users\where\my\data\store"
par_files = glob.glob(os.path.join(par_path, "*.parquet"))

ao_dpp = pd.read_parquet(r'C:\Users\path\where\my\data\store\fb_add-on_dpp_output.parquet')
ao_ag = pd.read_parquet(r'C:\Users\path\where\my\data\store\fb_add-on_agegender_output.parquet')

for file in par_files:
    df = pd.read_parquet(file)
    if 'dpp' in file:
        ## join file
        final_df = df.merge(ao_dpp, how='left', 
                                on=['account_id', 'account_name', 'campaign_id', 'campaign_name', 'ad_set_id', 'ad_set_name', 
                                'ad_id', 'ad_name', 'year', 'month', 'platform', 'device_platform', 'placement', 'objective'])
        final_df.rename(columns={'impressions_x': 'impressions', 'impressions_y': 'impressions_add-on', 'leads_x': 'leads', 'leads_y': 'leads_add-on'}, inplace=True)

        ## reindex file
        final_df = final_df.reindex(columns=['business_id', 'media_platform', 'agency_id', 'agency', 'account_id', 'account_name', 'campaign_id', 'campaign_name', 'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name', 'year', 'month',
                                            'platform', 'device_platform', 'placement', 'advertiser', 'industry', 'objective', 'currency', 'creative_object_type',
                                            'impressions','amount_spent', 'link_clicks', 'page_likes', '3-second_video_plays', 'landing_page_views', 'app_installs', 'meta_leads', 'mobile_app_adds_to_cart', 
                                            'website_adds_to_cart', 'mobile_app_content_views', 'website_content_views', 'mobile_app_purchases', 'website_purchases', 'mobile_app_adds_to_cart_conversion_value', 
                                            'website_adds_to_cart_conversion_value', 'mobile_app_purchases_conversion_value', 'website_purchases_conversion_value', 'leads', 'clicks_(all)', 'post_engagement', 
                                            'impressions_add-on', 'event_responses', 'outbound_clicks', 'leads_add-on', 'video_thruplay'])

        ## Export final result
        final_df.to_parquet(r'C:\Users\nsanva01\Desktop\Data Center\Facebook\final_dpp_result.parquet', index=False)

    if 'age-gender' in file:
        final_df = df.merge(ao_ag, how='left', 
                        left_on=['account_id', 'account_name', 'campaign_id', 'campaign_name', 'ad_set_id', 'ad_set_name', 
                        'ad_id', 'ad_name', 'year', 'month', 'gender', 'age', 'objective'], 
                        right_on=['account_id', 'account_name', 'campaign_id', 'campaign_name', 'ad_set_id', 'ad_set_name', 
                        'ad_id', 'ad_name', 'year', 'month', 'gender', 'age', 'objective'])
        final_df.rename(columns={'impressions_x': 'impressions', 'impressions_y': 'impressions_add-on', 'leads_x': 'leads', 'leads_y': 'leads_add-on'}, inplace=True)

        final_df = final_df.reindex(columns=['business_id', 'media_platform', 'agency_id', 'agency', 'account_id', 'account_name', 'campaign_id', 'campaign_name', 'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name', 'year', 'month',
                                    'gender', 'age', 'advertiser', 'industry', 'objective', 'currency', 'creative_object_type',
                                    'impressions','amount_spent', 'link_clicks', 'page_likes', '3-second_video_plays', 'landing_page_views', 'app_installs', 'meta_leads', 'mobile_app_adds_to_cart', 
                                    'website_adds_to_cart', 'mobile_app_content_views', 'website_content_views', 'mobile_app_purchases', 'website_purchases', 'mobile_app_adds_to_cart_conversion_value', 
                                    'website_adds_to_cart_conversion_value', 'mobile_app_purchases_conversion_value', 'website_purchases_conversion_value', 'leads', 'clicks_(all)', 'post_engagement', 
                                    'impressions_add-on', 'event_responses', 'outbound_clicks', 'leads_add-on', 'video_thruplay'])
        
        final_df.to_parquet(r'C:\Users\final\path\final_age-gender_result.parquet', index=False)